In [11]:
%pwd

'C:\\ML Projects\\Facial_Emotion_Recognition_End2End'

In [12]:
import os
from pathlib import Path
from dataclasses import dataclass  

In [13]:
os.chdir('C:\ML Projects\Facial_Emotion_Recognition_End2End')

In [14]:
%pwd

'C:\\ML Projects\\Facial_Emotion_Recognition_End2End'

In [15]:
@dataclass(frozen=True)
class PrepareBaseModel_Arguments:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int
    params_weights: str
    params_include_top: bool


In [16]:
from src.Facial_Emotion_Recognition.constant import *
from src.Facial_Emotion_Recognition.utils.common import read_yaml,create_directories
from src.Facial_Emotion_Recognition.logger import logging

In [17]:
class ConfigurationManager:           #it will prepare or extract the arguments 

    def __init__(                      
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH
            ):

            
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModel_Arguments:
        
        config=self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config= PrepareBaseModel_Arguments(

            base_model_path=config.base_model_path,
            root_dir=config.root_dir,
            updated_base_model_path=config.updated_base_model_path,
            params_classes=self.params.CLASSES,
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_learning_rate=self.params.LEARNING_RATE,
             
        )      

        logging.info('config of base model has prepared')

        return prepare_base_model_config    
                 

In [18]:
import os
from zipfile import ZipFile
from urllib.request import Request
import tensorflow as tf

In [19]:
class PrepareBaseModel:

    def __init__(self, config: PrepareBaseModel_Arguments):
        self.config=config

    @staticmethod
    def save_model( path: Path, model: tf.keras.Model):
        model.save(path)

    def get_base_model(self):

        self.model=tf.keras.applications.vgg16.VGG16(
            weights=self.config.params_weights,
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

        logging.info('base model has been saved')

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        
        if freeze_all :
            for layer in model.layers:
                model.trainable = False
        
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            activation='softmax',
            units=classes
        )(flatten_in)

        full_model=tf.keras.models.Model(

            inputs=model.input,
            outputs=prediction
            
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentrophy',
            metrics=['accuracy']
        )

        full_model.summary()

        logging.info

        return full_model
    

    # @staticmethod
    def updated_base_model(self):

        self.full_model=self.prepare_full_model(

            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )




In [20]:
try:
    config=ConfigurationManager()
    model_prepare=config.get_prepare_base_model_config()
    model=PrepareBaseModel(config=model_prepare)
    model.get_base_model()
    model.updated_base_model()

except Exception as e:
    raise e

[2023-10-14 19:04:29,125: INFO: common: yaml file : config\config.yaml loaded successfully]
[2023-10-14 19:04:29,135: INFO: common: yaml file : params.yaml loaded successfully]
[2023-10-14 19:04:29,140: INFO: common: created directories at :artifacts]
[2023-10-14 19:04:29,145: INFO: common: created directories at :artifacts/prepare_base_model]
[2023-10-14 19:04:29,145: INFO: 138849538: config of base model has prepared]
[2023-10-14 19:04:31,055: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


c:\Users\ACER\anaconda3\envs\venv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-10-14 19:04:31,353: INFO: 4162541766: base model has been saved]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                      